In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split as tts
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import Normalizer,StandardScaler
from cvgbm import run_gbm
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import RepeatedStratifiedKFold

from sklearn.tree import DecisionTreeClassifier as DTC
from sklearn.ensemble import StackingClassifier
from sklearn.metrics import classification_report,roc_auc_score

from xgboost import XGBClassifier as xgbc
from lightgbm import LGBMClassifier
import lightgbm as lgb
import h5py
import time

In [3]:
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns',500)

In [4]:
train=pd.read_csv('Ftrain.csv')
test=pd.read_csv('Ftest.csv')

In [5]:
type(train)

pandas.core.frame.DataFrame

In [6]:
target=train['Is_Lead'].astype(int)
train=train.drop('Is_Lead',axis=1)

In [7]:
features=train.columns.values

In [8]:
oversample=SMOTE(sampling_strategy=.55)
X,y=oversample.fit_resample(train,target)

In [9]:
num_cols=[ 'Vintage','Avg_Account_Balance','Re_Entrepreneur_Sum', 'Re_Salaried_Sum', 'Re_Self_Employed_Sum',
       'Re_Other_Sum', 'Re_Credit_Product_Sum', 'Re_Vintage_Avg',
       'Re_Vintage_skew', 'Re_Channel_X1_Sum', 'Re_Channel_X2_Sum',
       'Re_Channel_X3_Sum', 'Re_Channel_X4_Sum', 'Re_Mid_Age_Sum',
       'Re_Young_Sum', 'Re_Old_Sum', 'Re_Active', 'Re_Balance_Avg',
       'Re_Balance_skew', 'Re_Balance_std', 'Re_Balance_top',
       'Re_Balance_bottom', 'Balance_to_Age', 'Vintage_to_Age']

In [38]:
params={'random_state':100,'objective':'binary:logistic','n_estimators':2714,'booster':'gbtree','eta':.0215,'min_child_weight':20,'colsample_bytree':.75,'scale_pos_weight':2.25,'reg_lambda':17.789,'max_depth':5}

xclf=xgbc(**params)



fit_params={'eval_metric':['logloss','auc'],'verbose':500}

xgb_oofs,xgb_pred=run_gbm(clf=xclf,train=X,target=y,test=test,features=features,num_features=num_cols,fit_params=fit_params)

------------------------------------Fold1------------------------------------
[0]	validation_0-logloss:0.68471	validation_0-auc:0.84571	validation_1-logloss:0.68465	validation_1-auc:0.84611
[500]	validation_0-logloss:0.39020	validation_0-auc:0.90543	validation_1-logloss:0.38941	validation_1-auc:0.90425
[1000]	validation_0-logloss:0.36979	validation_0-auc:0.91385	validation_1-logloss:0.37167	validation_1-auc:0.91082
[1500]	validation_0-logloss:0.36142	validation_0-auc:0.91758	validation_1-logloss:0.36541	validation_1-auc:0.91306
[2000]	validation_0-logloss:0.35578	validation_0-auc:0.92035	validation_1-logloss:0.36178	validation_1-auc:0.91448
[2500]	validation_0-logloss:0.35088	validation_0-auc:0.92286	validation_1-logloss:0.35885	validation_1-auc:0.91557
[2713]	validation_0-logloss:0.34974	validation_0-auc:0.92353	validation_1-logloss:0.35829	validation_1-auc:0.91579
ROC score for Fold 1 Validation :0.9157386365727906 
ROC score for Validation :0.9131392876356051 
----------------------

[1500]	validation_0-logloss:0.36325	validation_0-auc:0.91681	validation_1-logloss:0.37134	validation_1-auc:0.91399
[2000]	validation_0-logloss:0.35700	validation_0-auc:0.91979	validation_1-logloss:0.36703	validation_1-auc:0.91551
[2500]	validation_0-logloss:0.35253	validation_0-auc:0.92205	validation_1-logloss:0.36436	validation_1-auc:0.91651
[2713]	validation_0-logloss:0.35064	validation_0-auc:0.92302	validation_1-logloss:0.36322	validation_1-auc:0.91695
ROC score for Fold 9 Validation :0.9169480744777319 
ROC score for Validation :0.9129532534248785 
------------------------------------Fold10------------------------------------
[0]	validation_0-logloss:0.68471	validation_0-auc:0.84580	validation_1-logloss:0.68471	validation_1-auc:0.84505
[500]	validation_0-logloss:0.39043	validation_0-auc:0.90569	validation_1-logloss:0.39390	validation_1-auc:0.90134
[1000]	validation_0-logloss:0.36980	validation_0-auc:0.91417	validation_1-logloss:0.37592	validation_1-auc:0.90866
[1500]	validation_0-l

In [ ]:
xgb_oofs=pd.DataFrame([xgb_oofs],index=['lgbm_oofs']).T
xgb_pred=pd.DataFrame([xgb_pred],index=['xgb_pred']).T
xgb_oofs.to_csv('xgb_oofs.csv',index=False)
xgb_pred.to_csv('xgb_pred.csv',index=False)

## LGBM

In [29]:
params={'n_estimators':4209,'objective':'binary','metric':['logloss','auc'],'random_state':100,'learning_rate':.02,'min_child_samples':300,'max_bin':50,'colsample_bytree':.5,'max_depth':5,'num_leaves':127,'reg_lambda':22.54,'reg_alpha':10}
model=LGBMClassifier(**params)
fit_params={'eval_metric':['logloss','auc'],'verbose':500}

lgbm_oofs,lgbm_pred=run_gbm(clf=model,train=X,target=y,test=test,features=features,num_features=num_cols,fit_params=fit_params)

------------------------------------Fold1------------------------------------
[500]	training's binary_logloss: 0.363875	training's auc: 0.89632	valid_1's binary_logloss: 0.364143	valid_1's auc: 0.895565
[1000]	training's binary_logloss: 0.346665	training's auc: 0.903964	valid_1's binary_logloss: 0.34825	valid_1's auc: 0.902426
[1500]	training's binary_logloss: 0.339093	training's auc: 0.907689	valid_1's binary_logloss: 0.341984	valid_1's auc: 0.905146
[2000]	training's binary_logloss: 0.334811	training's auc: 0.909827	valid_1's binary_logloss: 0.338684	valid_1's auc: 0.906467
[2500]	training's binary_logloss: 0.331955	training's auc: 0.911339	valid_1's binary_logloss: 0.336694	valid_1's auc: 0.907189
[3000]	training's binary_logloss: 0.329908	training's auc: 0.912561	valid_1's binary_logloss: 0.335435	valid_1's auc: 0.907685
[3500]	training's binary_logloss: 0.328262	training's auc: 0.913551	valid_1's binary_logloss: 0.334618	valid_1's auc: 0.907948
[4000]	training's binary_logloss: 0.

ROC score for Fold 7 Validation :0.9080807531783285 
ROC score for Validation :0.9054524967844393 
------------------------------------Fold8------------------------------------
[500]	training's binary_logloss: 0.363755	training's auc: 0.896515	valid_1's binary_logloss: 0.364226	valid_1's auc: 0.895368
[1000]	training's binary_logloss: 0.346864	training's auc: 0.903971	valid_1's binary_logloss: 0.348831	valid_1's auc: 0.901437
[1500]	training's binary_logloss: 0.339459	training's auc: 0.907722	valid_1's binary_logloss: 0.342762	valid_1's auc: 0.904076
[2000]	training's binary_logloss: 0.335043	training's auc: 0.909843	valid_1's binary_logloss: 0.339352	valid_1's auc: 0.905476
[2500]	training's binary_logloss: 0.332234	training's auc: 0.911318	valid_1's binary_logloss: 0.337332	valid_1's auc: 0.906393
[3000]	training's binary_logloss: 0.330202	training's auc: 0.912422	valid_1's binary_logloss: 0.336049	valid_1's auc: 0.906962
[3500]	training's binary_logloss: 0.328484	training's auc: 0.9

In [30]:
lgbm_oofs=pd.DataFrame([lgbm_oofs],index=['lgbm_oofs']).T

In [31]:
lgbm_pred=pd.DataFrame([lgbm_pred],index=['lgbm_pred']).T

In [32]:
lgbm_oofs.to_csv('lgbm_oofs.csv',index=False)
lgbm_pred.to_csv('lgbm_pred.csv',index=False)

## Final Prediction

In [43]:
final_pred=(lgbm_pred['lgbm_pred']+xgb_pred['xgb_pred'])/2

In [44]:
final_pred

0         0.000280
1         0.653321
2         0.000260
3         0.000203
4         0.000295
            ...   
105307    0.987700
105308    0.630572
105309    0.849447
105310    0.412425
105311    0.000517
Length: 105312, dtype: float64